### Постановка задачи

Необходимо предсказать биологический ответ молекул (столбец 'Activity') по их химическому составу (столбцы D1-D1776).

Данные представлены в формате CSV. Каждая строка представляет молекулу.

Первый столбец Activity содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1];
Остальные столбцы D1-D1776 представляют собой молекулярные дескрипторы — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.
Предварительная обработка не требуется, данные уже закодированы и нормализованы.

В качестве метрики будем использовать F1-score.

Необходимо обучить две модели: логистическую регрессию и случайный лес. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.

In [66]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные модели
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn.model_selection import cross_val_score #Кросс валидация

# Импорт оптимизаторов параметров
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import hyperopt
from hyperopt import hp, fmin, tpe, Trials

import optuna

%matplotlib inline
plt.style.use('seaborn')

In [67]:
data = pd.read_csv('data/_train_sem09.csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [68]:
X = data.drop(['Activity'], axis=1)
y = data['Activity']

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

# Мое решение

Сначала получим значения меток F1-score для логистической регресии и случайного леса без использования методов оптимизации, чтобы иметь в дальнейшем базу для сравнения.

Для обобщения результатов создаем два словаря logreg и ranfor, в которые будем заносить результаты обучения каждым методом. 

In [70]:
logreg = {}
ranfor = {}

### Логистическая регрессия (без методов оптимизации)

In [71]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(random_state=42, max_iter = 50)
#Обучаем модель
log_reg.fit(X_train, y_train)

y_test_pred = log_reg.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.78


C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [72]:
logreg['LogReg_init'] = round(metrics.f1_score(y_test, y_test_pred), 3)
logreg['LogReg_init']

0.782

### Случайный лес (без методов оптимизации)

In [73]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=42)
#Обучаем модель
rf.fit(X_train, y_train)

y_test_pred = rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


In [74]:
ranfor['RanFor_init'] = round(metrics.f1_score(y_test, y_test_pred), 3)
ranfor['RanFor_init']

0.817

Далее последовательно используются методы оптимизации

# Метод GridSeachCV

### Логистическая регрессия, метод GridSeachCV

In [75]:
param_grid = [
    {'penalty' : ['l2', 'none'], # тип регуляризации
    'solver' : ['newton-cg', 'lbfgs', 'sag'], # алгоритм оптимизации
    'C' : [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации
    # другой набор гиперпараметров
    {'penalty': ['l1', 'l2'] ,
    'solver': ['liblinear', 'saga'],
    'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]

grid_search_LR = GridSearchCV(
    estimator = linear_model.LogisticRegression(random_state= 42, max_iter = 50),
    param_grid = param_grid,
    cv = 5,
    n_jobs = -1,
    scoring='f1'
)
%time grid_search_LR.fit(X_train, y_train)

y_test_pred = grid_search_LR.predict(X_test)

print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_LR.best_params_))

CPU times: total: 8.08 s
Wall time: 8min 35s
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'saga'}


C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [76]:
logreg['LogReg_GridSeach'] = round(metrics.f1_score(y_test, y_test_pred), 3)
logreg['LogReg_GridSeach']

0.794

### Случайный лес, метод GridSeachCV

In [77]:
param_distributions = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': list(np.linspace(5, 25, 5, dtype=int)),
              'max_depth': list(np.linspace(1, 40, 5, dtype=int))
              }
            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 50, 
    n_jobs = -1,
    scoring = 'f1'
)  
%time random_search_forest.fit(X_train, y_train) 

y_test_pred = random_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

CPU times: total: 3.69 s
Wall time: 58.4 s
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'n_estimators': 170, 'min_samples_leaf': 5, 'max_depth': 30}


In [78]:
ranfor['RanFor_GridSeach'] = round(metrics.f1_score(y_test, y_test_pred), 3)
ranfor['RanFor_GridSeach']

0.817

# Метод RandomizedSeachCV

### Логистическая регрессия, метод RandomizedSeachCV

In [79]:
param_distributions = [
                        {'penalty': ['l2', 'none'] , # тип регуляризации
                         'solver' : ['newton-cg', 'lbfgs', 'sag'], # алгоритм оптимизации
                          'C': list(np.linspace(0.01, 1, 10, dtype=float))
                        },
                        {'penalty': ['l1', 'l2'] ,
                         'solver': ['liblinear', 'saga'],
                          'C': list(np.linspace(0.01, 1, 10, dtype=float))
                        }]
random_search_LR = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter= 50), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 20, 
    n_jobs = -1,
    scoring='f1'
)  
%time random_search_LR.fit(X_train, y_train) 

y_test_pred = random_search_LR.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_LR.best_params_))

CPU times: total: 3.42 s
Wall time: 3min 7s
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'solver': 'saga', 'penalty': 'l2', 'C': 0.12}


C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [80]:
logreg['LogReg_RandSeach'] = round(metrics.f1_score(y_test, y_test_pred), 3)
logreg['LogReg_RandSeach']

0.794

### Случайный лес, метод RandomizedSeachCV

In [81]:
param_distributions = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': list(np.linspace(5, 25, 5, dtype=int)),
              'max_depth': list(np.linspace(1, 40, 5, dtype=int))
              }
            
random_search_RF = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 50, 
    n_jobs = -1,
    scoring= 'f1'
)  
%time random_search_RF.fit(X_train, y_train) 

y_test_pred = random_search_RF.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_RF.best_params_))

CPU times: total: 2.94 s
Wall time: 58 s
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'n_estimators': 140, 'min_samples_leaf': 5, 'max_depth': 40}


In [82]:
ranfor['RanFor_RandSeach'] = round(metrics.f1_score(y_test, y_test_pred), 3)
ranfor['RanFor_RandSeach']

0.815

# Метод Hyperopt

In [83]:
# Уточняем текущую версию HyperOpt
print("Версия Hyperopt : {}".format(hyperopt.__version__))

Версия Hyperopt : 0.2.5


### Логистическая регрессия, метод HyperOpt

In [84]:
space ={
    'penalty' : hp.choice(label='penalty', options= ['l1', 'l2']), # тип регуляризации
    'solver' : hp.choice(label = 'solver', options= ['liblinear', 'saga']), # алгоритм оптимизации
    'C' : hp.loguniform(label='C', low=-2*np.log(10), high=2*np.log(10)), # уровень силы регурялизации
    'max_iter': hp.choice('max_iter', [50])}

In [85]:
random_state = 42
def hyperopt_logr(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    params = {'penalty': params['penalty'],
              'solver': params['solver'],
              'C': params['C'],
              'max_iter': params['max_iter']
              }
    model_HO = linear_model.LogisticRegression(**params, random_state=random_state)

    # обучаем модель
    model_HO.fit(X, y)
    score = cross_val_score(model_HO, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [86]:
%%time
# начинаем подбор гиперпараметров
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_logr, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=50, # максимальное количество итераций
          trials=trials, # логирование результатов
          # rstate = np.random.default_rng(42) # для версии 0.2.7
          rstate=np.random.RandomState(42) # (random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

  0%|                                                                           | 0/50 [00:00<?, ?trial/s, best loss=?]

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



  8%|███▊                                            | 4/50 [00:44<09:22, 12.23s/trial, best loss: -0.7644504520739478]

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 10%|████▊                                           | 5/50 [00:52<07:57, 10.62s/trial, best loss: -0.7644504520739478]

...

 94%|█████████████████████████████████████████████   | 47/50 [07:11<00:39, 13.16s/trial, best loss: -0.777617316199278]

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 98%|███████████████████████████████████████████████ | 49/50 [07:22<00:08,  8.85s/trial, best loss: -0.777617316199278]

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



100%|████████████████████████████████████████████████| 50/50 [07:31<00:00,  9.02s/trial, best loss: -0.777617316199278]
Наилучшие значения гиперпараметров {'C': 0.7038661632470573, 'max_iter': 0, 'penalty': 0, 'solver': 1}
CPU times: total: 3min 18s
Wall time: 7min 31s


Поскольку данная версия hyperopt в качестве индексов массива выдает для логистической регрессии только индексы массива, то воспользуемся space_eval, чтобы обращаться к элементам массива напрямую

In [87]:
from hyperopt import space_eval
hyperparams = space_eval(space, best)
hyperparams

{'C': 0.7038661632470573, 'max_iter': 50, 'penalty': 'l1', 'solver': 'saga'}

In [88]:
# рассчитаем точность для тестовой выборки
model_HO = linear_model.LogisticRegression(
    random_state=random_state,
    penalty = hyperparams['penalty'],
    solver = hyperparams['solver'],
    C = hyperparams['C'],
    max_iter = hyperparams['max_iter']
    )
model_HO.fit(X_train, y_train)

y_test_pred = model_HO.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.80


C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [89]:
logreg['LogReg_HO'] = round(metrics.f1_score(y_test, y_test_pred), 3)
logreg['LogReg_HO']

0.796

### Случайный лес, метод HyperOpt

In [90]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 80, 200, 1),
       'max_depth' : hp.quniform('max_depth', 1, 30, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 5, 25, 1)
      }

In [91]:
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
    }
    # используем эту комбинацию для построения модели   
    model_HO_RF = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model_HO_RF.fit(X, y)
    score = cross_val_score(model_HO_RF, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    return -score  

In [92]:
%%time
# начинаем подбор гиперпараметров
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=50, # максимальное количество итераций
          trials=trials, # логирование результатов
          # rstate = np.random.default_rng(42) # для версии 0.2.7
          rstate=np.random.RandomState(42) # (random_state)# для версии 0.2.5, фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|███████████████████████████████████████████████| 50/50 [02:48<00:00,  3.37s/trial, best loss: -0.8005102732363687]
Наилучшие значения гиперпараметров {'max_depth': 30.0, 'min_samples_leaf': 5.0, 'n_estimators': 115.0}
CPU times: total: 1min 11s
Wall time: 2min 48s


In [93]:
# рассчитаем точность для тестовой выборки
model_HO_RF = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)

model_HO_RF.fit(X_train, y_train)

y_test_pred = model_HO_RF.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


In [94]:
ranfor['RanFor_HO'] = round(metrics.f1_score(y_test, y_test_pred), 3)
ranfor['RanFor_HO']

0.817

# Метод Optuna

In [95]:
print("Версия Optuna: {}".format(optuna.__version__))

Версия Optuna: 3.0.2


### Логистическая регрессия, метод Optuna 

In [96]:
random_state=42
def optuna_lr(trial):
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])
    C = trial.suggest_float(name='C', low=0.01, high=1.01, step = 0.05)
   
    ## Создаем модель
    model_Op_LR = linear_model.LogisticRegression(
                                    penalty=penalty,
                                    solver=solver,
                                    C=C,
                                    random_state=random_state
                              )
    ## Обучаем модель
    model_Op_LR.fit(X_train, y_train)
    score = cross_val_score(model_Op_LR, X_train, y_train, cv=5, scoring="f1", n_jobs=-1).mean()
    
    return score

In [97]:
%%time
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
study.optimize(optuna_lr, n_trials=50)

[I 2022-10-28 19:13:29,492] A new study created in memory with name: LogisticRegression
C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2022-10-28 19:13:40,161] Trial 0 finished with value: 0.763296420693192 and parameters: {'penalty': 'l2', 'solver': 'saga', 'C': 0.7100000000000001}. Best is trial 0 with value: 0.763296420693192.
C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2022-10-28 19:13:50,154] Trial 1 finished with value: 0.7643926177712026 and parameters: {'penalty': 'l2', 'solver': 'saga', 'C': 0.46}. Best is trial 1 with value: 0.7643926177712026.
C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2022-10-28 19:18:15,957] Trial 24 finished with value: 0.7774420512902167 and parameters: {'penalty': 'l1', 'solver': 'saga', 'C': 0.16000000000000003}. Best is trial 19 with value: 0.7774420512902167.
[I 2022-10-28 19:18:18,984] Trial 25 finished with value: 0.74701333922839 and parameters: {'penalty': 'l1', 'solver': 'saga', 'C': 0.01}. Best is trial 19 with value: 0.7774420512902167.
[I 2022-10-28 19:18:20,320] Trial 26 finished with value: 0.7754745853055905 and parameters: {'penalty': 'l1', 'solver': 'liblinear', 'C': 0.26}. Best is trial 19 with value: 0.7774420512902167.
C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2022-10-28 19:18:34,931] Trial 27 finished with value: 

[I 2022-10-28 19:22:54,825] Trial 48 finished with value: 0.7779282846241222 and parameters: {'penalty': 'l2', 'solver': 'liblinear', 'C': 0.060000000000000005}. Best is trial 48 with value: 0.7779282846241222.
[I 2022-10-28 19:22:56,183] Trial 49 finished with value: 0.7779282846241222 and parameters: {'penalty': 'l2', 'solver': 'liblinear', 'C': 0.060000000000000005}. Best is trial 48 with value: 0.7779282846241222.


CPU times: total: 4min 4s
Wall time: 9min 26s


In [98]:
# выводим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров {}".format(study.best_params))

Наилучшие значения гиперпараметров {'penalty': 'l2', 'solver': 'liblinear', 'C': 0.060000000000000005}


In [99]:
model_Op_LR = linear_model.LogisticRegression(**study.best_params, random_state=random_state,)

model_Op_LR.fit(X_train, y_train)

y_test_pred = model_Op_LR.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.79


In [100]:
logreg['LogReg_Optuna'] = round(metrics.f1_score(y_test, y_test_pred), 3)
logreg['LogReg_Optuna']

0.792

### Случайный лес, метод Optuna

In [101]:
random_state = 42
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 400, 1)  #100, 200, 1
  max_depth = trial.suggest_int('max_depth', 5, 30, 1)     # 10, 30, 1
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 12, 1)

  # создаем модель
  model_Op_rf = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model_Op_rf.fit(X_train, y_train)
  score = cross_val_score(model_Op_rf, X_train, y_train, cv = 5, scoring = 'f1', n_jobs= -1).mean()
    
  return score

In [102]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=50)

[I 2022-10-28 19:24:55,516] A new study created in memory with name: RandomForestClassifier
[I 2022-10-28 19:25:04,566] Trial 0 finished with value: 0.7874215084755555 and parameters: {'n_estimators': 397, 'max_depth': 15, 'min_samples_leaf': 11}. Best is trial 0 with value: 0.7874215084755555.
[I 2022-10-28 19:25:11,206] Trial 1 finished with value: 0.7934614021998577 and parameters: {'n_estimators': 257, 'max_depth': 23, 'min_samples_leaf': 8}. Best is trial 1 with value: 0.7934614021998577.
[I 2022-10-28 19:25:16,602] Trial 2 finished with value: 0.7938515062539839 and parameters: {'n_estimators': 237, 'max_depth': 8, 'min_samples_leaf': 2}. Best is trial 2 with value: 0.7938515062539839.
[I 2022-10-28 19:25:23,411] Trial 3 finished with value: 0.7906509830223318 and parameters: {'n_estimators': 282, 'max_depth': 30, 'min_samples_leaf': 12}. Best is trial 2 with value: 0.7938515062539839.
[I 2022-10-28 19:25:34,084] Trial 4 finished with value: 0.8012512162468102 and parameters: {'n

[I 2022-10-28 19:30:55,296] Trial 38 finished with value: 0.8057827759838958 and parameters: {'n_estimators': 244, 'max_depth': 14, 'min_samples_leaf': 3}. Best is trial 36 with value: 0.8138336806772921.
[I 2022-10-28 19:31:01,788] Trial 39 finished with value: 0.7847305581129758 and parameters: {'n_estimators': 301, 'max_depth': 9, 'min_samples_leaf': 12}. Best is trial 36 with value: 0.8138336806772921.
[I 2022-10-28 19:31:08,626] Trial 40 finished with value: 0.7972188501629959 and parameters: {'n_estimators': 226, 'max_depth': 16, 'min_samples_leaf': 5}. Best is trial 36 with value: 0.8138336806772921.
[I 2022-10-28 19:31:18,859] Trial 41 finished with value: 0.8141075658656834 and parameters: {'n_estimators': 307, 'max_depth': 15, 'min_samples_leaf': 2}. Best is trial 41 with value: 0.8141075658656834.
[I 2022-10-28 19:31:27,706] Trial 42 finished with value: 0.8105721223945144 and parameters: {'n_estimators': 273, 'max_depth': 14, 'min_samples_leaf': 2}. Best is trial 41 with va

CPU times: total: 3min 14s
Wall time: 7min 28s


In [103]:
# выводим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров {}".format(study.best_params))

Наилучшие значения гиперпараметров {'n_estimators': 307, 'max_depth': 15, 'min_samples_leaf': 2}


In [104]:
# рассчитаем точность для тестовой выборки
model_Op_rf = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model_Op_rf.fit(X_train, y_train)
y_test_pred = model_Op_rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.83


In [105]:
ranfor['RanFor_Optuna'] = round(metrics.f1_score(y_test, y_test_pred), 3)
ranfor['RanFor_Optuna']

0.825

In [106]:
# преобразуем словари  в датафреймы    
df_logreg = pd.DataFrame([logreg])
display(df_logreg)

df_ranfor = pd.DataFrame([ranfor])
display(df_ranfor)

,LogReg_init,LogReg_GridSeach,LogReg_RandSeach,LogReg_HO,LogReg_Optuna
0,0.782,0.794,0.794,0.796,0.792


,RanFor_init,RanFor_GridSeach,RanFor_RandSeach,RanFor_HO,RanFor_Optuna
0,0.817,0.817,0.815,0.817,0.825


### Некоторые выводы

1. Модель, использующая случайный лес, показывает более высокие результаты, чем модель, использующая логистическую регрессию при использовнии любого метода оптимизации
2. Наилучшие результаты для логистической регрессии показал метод HyperOpt (0.796), а для случайного леса - Optuna (0.825)